### Installing some tools

In [38]:
!pip install selenium

In [39]:
!pip install gspread

In [40]:
!pip install oauth2client

Install chrome driver if you run from local: https://chromedriver.chromium.org/downloads

### Importing Libraries

In [41]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

### Scraping Game-lines Data

Important: You may need to run this cell more than once for it to work.

In [43]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://bovada.lv/es/sports/baseball/mlb?overlay=register')
time.sleep(5)

grouped_events = driver.find_element('xpath', '(//div[@class="next-events-bucket"]//div[@class="grouped-events"])[1]')

teams = grouped_events.find_elements('xpath', './/span[@class="name"]')

teams_list = []

for team in teams:
    teams_list.append(team.text)

teams_away = teams_list[::2]
teams_home = teams_list[1::2]

scores = grouped_events.find_elements('xpath', './/span[@class="market-line bet-handicap both-handicaps"]')

total = []

for score in scores:
    total.append(score.text)

for team_away, team_home, score in zip(teams_away, teams_home, total):
    print(f"{team_away}: Away")
    print(f"{team_home}: Home")
    print(f"Total: {score}")
    print()

driver.quit()

Pittsburgh Pirates: Away
San Diego Padres: Home
Total: 8.5

St. Louis Cardinals: Away
Arizona Diamondbacks: Home
Total: 8.5

Toronto Blue Jays: Away
Los Angeles Dodgers: Home
Total: 10.5



### Creating Game-lines DataFrame

In [44]:
data = list(zip(teams_away, teams_home, total))

gamelines_df = pd.DataFrame(data, columns=['Away', 'Home', 'Total'])
gamelines_df

,Away,Home,Total
0,Pittsburgh Pirates,San Diego Padres,8.5
1,St. Louis Cardinals,Arizona Diamondbacks,8.5
2,Toronto Blue Jays,Los Angeles Dodgers,10.5


### Scraping 5-Inning-lines Data

Important: You may need to run this cell more than once for it to work.

In [45]:
chrome_options1 = Options()
driver1 = webdriver.Chrome(options=chrome_options1)
driver1.get('https://www.bovada.lv/es/sports/beisbol/lineas-para-las-primeras-5-entradas-cuotas?overlay=register') ### if your browser is in english
                                                                                        ### you should change the link to this:
                                                                                        ### https://bovada.lv/sports/baseball/mlb/5-inning-lines-odd?overlay=register
time.sleep(5)

grouped_events1 = driver1.find_element('xpath', '(//div[@class="next-events-bucket"]//div[@class="grouped-events"])[1]')

teams1 = grouped_events1.find_elements('xpath', './/span[@class="name"]')

teams_list1 = []

for team in teams1:
    teams_list1.append(team.text)

teams_away1 = teams_list1[::2]
teams_home1 = teams_list1[1::2]

scores1 = grouped_events1.find_elements('xpath', './/span[@class="market-line bet-handicap both-handicaps"]')

total1 = []

for score in scores1:
    total1.append(score.text)

for team_away, team_home, score in zip(teams_away1, teams_home1, total1):
    print(f"{team_away}: Away")
    print(f"{team_home}: Home")
    print(f"Total: {score}")
    print()

driver1.quit()

Pittsburgh Pirates: Away
San Diego Padres: Home
Total: 4.5

St. Louis Cardinals: Away
Arizona Diamondbacks: Home
Total: 4.5

Toronto Blue Jays: Away
Los Angeles Dodgers: Home
Total: 6.0



### Creating 5-Inning-lines DataFrame

In [46]:
data1 = list(zip(teams_away1, teams_home1, total1))

inninglines_df = pd.DataFrame(data1, columns=['Away', 'Home', 'Total'])
inninglines_df

,Away,Home,Total
0,Pittsburgh Pirates,San Diego Padres,4.5
1,St. Louis Cardinals,Arizona Diamondbacks,4.5
2,Toronto Blue Jays,Los Angeles Dodgers,6.0


### Move the DataFrames to a spreadsheet

In [61]:
# Service authentication data
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('sportsbookscraping-da08350e1110.json', scope)

# Authenticate the service account
gc = gspread.authorize(credentials)

# Opens the spreadsheet
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/18ulEvud026DINujAW1e1nzgyjj-m-XT0PvNemaydhAc/edit#gid=0')

# Get the first sheet of the file (you can change the index if there are multiple sheets)
worksheet = sh.get_worksheet(0)

# Clear the existing data in the worksheet
worksheet.clear()

dataframe1 = gamelines_df
dataframe2 = inninglines_df

data_gamelines = [dataframe1.columns.tolist()] + dataframe1.values.tolist()
data_inninglines = [dataframe2.columns.tolist()] + dataframe2.values.tolist()

# Write the data to the spreadsheet
worksheet.insert_rows(data_gamelines, 1)
worksheet.insert_rows(data_inninglines, len(data_gamelines) + 2)

# Apply formatting to the entire range of cells
cell_range = worksheet.range(f"A1:{chr(ord('A') + len(data_gamelines[0]) - 1)}{len(data_gamelines) + len(data_inninglines) + 1}")

# Update the worksheet with the modified cell properties
worksheet.update_cells(cell_range)

print("DataFrames have been successfully moved to Google Sheets.")

DataFrames have been successfully moved to Google Sheets.
